# 1. Cuestionario del alumno

In [1]:
from scipy import spatial
import numpy as np

In [2]:
BD = "proyecto_final"

In [3]:
dic_campos_bd = {
     1: "arte_diseno_arquitectura",
     2: "agricultura_forestal",
     3: "empresariales_administracion",
     4: "economicas",
     5: "educacion",
     6: "ingenieria_tecnologia",
     7: "historia_filosofia_teologia",
     8: "leyes_abogacia",
     9: "ciencias_salud",
    10: "ciencias_naturales",
    11: "ciencias_sociales",
    12: "otros",
}

dic_campos_usuario = {
    "arte_diseno_arquitectura": "Arte, diseño y arquitectura",
    "agricultura_forestal": "Agricultura y forestal",
    "empresariales_administracion": "Empresariales y administración",
    "economicas": "Economía",
    "educacion": "Educación",
    "ingenieria_tecnologia": "Ingeniería y tecnología",
    "historia_filosofia_teologia": "Historia, filosofía y teología",
    "leyes_abogacia": "Leyes y abogacía",
    "ciencias_salud": "Ciencias de la salud",
    "ciencias_naturales": "Ciencias naturales",
    "ciencias_sociales": "Ciencias sociales",
    "otros": "Otros",
}

In [4]:
print("Introduce el identificador de 3 campos de tu interés de mayor a menor preferencia:")
for key, value in dic_campos_bd.items():
    print(f"{'{:3d}'.format(key)}. {dic_campos_usuario[value]}")
print()

completed = False
selected_ids = []
max_length = 3
while not completed:
    selection = input(f"Introduce el identificador de tu elección ({len(selected_ids)}/{max_length}): ")
    if selection not in selected_ids:
        print(f"    Has elegido {dic_campos_usuario[dic_campos_bd[int(selection)]]}")
        selected_ids.append(selection)
        if len(selected_ids) == max_length:
            completed = True
    else:
        print("    Error: Ya has escogido ese campo")

Introduce el identificador de 3 campos de tu interés de mayor a menor preferencia:
  1. Arte, diseño y arquitectura
  2. Agricultura y forestal
  3. Empresariales y administración
  4. Economía
  5. Educación
  6. Ingeniería y tecnología
  7. Historia, filosofía y teología
  8. Leyes y abogacía
  9. Ciencias de la salud
 10. Ciencias naturales
 11. Ciencias sociales
 12. Otros

Introduce el identificador de tu elección (0/3): 4
    Has elegido Economía
Introduce el identificador de tu elección (1/3): 5
    Has elegido Educación
Introduce el identificador de tu elección (2/3): 11
    Has elegido Ciencias sociales


In [17]:
proj_usuario = [10, 5, 2]
print(selected_ids)

['4', '5', '11']


Sentencia para obtener el ranking de las universidades en los campos elegidos por el usuario

In [6]:
sentencia_select = f"SELECT {BD}.universidades.nombre AS nombre_univ, {BD}.campos.nombre AS nombre_campo, {BD}.rankings.id_univ, {BD}.rankings.id_campo, {BD}.rankings.puntuacion FROM {BD}.campos"
sentencia_join = f"""INNER JOIN {BD}.rankings ON {BD}.campos.id = {BD}.rankings.id_campo
                     INNER JOIN {BD}.universidades ON {BD}.universidades.id = {BD}.rankings.id_univ"""
ids_str = ','.join(selected_ids)
sentencia_where = f"WHERE {BD}.campos.id IN ({','.join(selected_ids)})"

sentencia_sql = f"""{sentencia_select}
                    {sentencia_join}
                    {sentencia_where}"""


rankings_df = spark.sql(sentencia_sql).fillna(0)
rankings_df.show()

2023-07-26 00:39:53,827 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-07-26 00:39:53,828 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2023-07-26 00:39:55,326 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+--------------------+-----------------+-------+--------+----------+
|         nombre_univ|     nombre_campo|id_univ|id_campo|puntuacion|
+--------------------+-----------------+-------+--------+----------+
|Universidad Nacio...|       economicas|     36|       4|         5|
|Universidad Nacio...|        educacion|     36|       5|         4|
|Universidad Nacio...|ciencias_sociales|     36|      11|        13|
|Universidad Obert...|       economicas|     37|       4|         4|
|Universidad Obert...|        educacion|     37|       5|         9|
|Universidad Obert...|ciencias_sociales|     37|      11|        39|
|Universidad Polit...|       economicas|     41|       4|         1|
|Universidad Polit...|        educacion|     41|       5|         3|
|Universidad Polit...|ciencias_sociales|     41|      11|         6|
|Universidad de Ba...|       economicas|     57|       4|        23|
|Universidad de Ba...|        educacion|     57|       5|        21|
|Universidad de Ba...|ciencias_soc

In [7]:
list_of_unis = zip(*(iter(rankings_df.collect()),) * 3)

In [8]:
unis_dict = {}
for campo_1, campo_2, campo_3 in list_of_unis:
    uni_dict = {campo_1['id_campo']: campo_1['puntuacion'],
                campo_2['id_campo']: campo_2['puntuacion'],
                campo_3['id_campo']: campo_3['puntuacion'],
                }
    unis_dict[campo_1['id_univ']] = uni_dict

In [79]:
#SEGUNDA PARTE DE LA PROYECCION BASAR EN EL LOGARITMO EN BASE A LA UNIVERSIDAD CON MAYOR NUMERO DE CARRERAS EN ESE
sentencia_select = f"SELECT {BD}.universidades.id AS id_univ, {BD}.universidades.nombre AS nombre_univ, {BD}.campos.nombre AS nombre_campo, COUNT(*) AS carreras FROM {BD}.campos"
sentencia_join = f"""INNER JOIN {BD}.facultades ON {BD}.campos.id = {BD}.facultades.id_campo INNER JOIN {BD}.grados ON {BD}.grados.id_fac = {BD}.facultades.id INNER JOIN {BD}.universidades ON {BD}.universidades.id = {BD}.facultades.id_univ"""

ids_str = ','.join(selected_ids)
sentencia_where = f"WHERE {BD}.campos.id IN ({selected_ids[0]})"
sentencia_group_by = f"GROUP BY {BD}.campos.nombre, {BD}.universidades.nombre, {BD}.universidades.id"
sentencia_sql = f"""{sentencia_select} {sentencia_join} {sentencia_where} {sentencia_group_by}"""


carreras_total_df = spark.sql(sentencia_sql)
carreras_total_df.show()


+-------+--------------------+------------+--------+
|id_univ|         nombre_univ|nombre_campo|carreras|
+-------+--------------------+------------+--------+
|     18|Universidad Compl...|  economicas|       2|
|     17|Universidad Catól...|  economicas|       5|
|     30|Universidad Inter...|  economicas|       1|
|     28|Universidad Inter...|  economicas|       2|
|     79|Universidad de Se...|  economicas|       2|
|     33|Universidad Jaume...|  economicas|       8|
+-------+--------------------+------------+--------+



In [54]:

print(sentencia_sql)
SELECT universidades.nombre AS nombre_univ, campos.nombre AS nombre_campo, COUNT(*) AS carreras FROM campos INNER JOIN facultades ON campos.id = facultades.id_campo INNER JOIN grados ON grados.id_fac = facultades.id INNER JOIN universidades ON universidades.id = facultades.id_univ WHERE campos.id IN (4,5,11) GROUP BY campos.nombre, universidades.nombre

SyntaxError: invalid syntax (3252934731.py, line 2)

In [20]:
def calculate_uni_projection(uni_dict, selected_ids):
    """
    """
    total_investigators = 0
    for investigators in uni_dict.values():
        total_investigators += investigators
    
    if total_investigators == 0:
        return [-100, -100, -100]
    # For each field, ordered by user preference, calculate the percentage of total investigators
    
    ordered_dict = sorted(uni_dict.items(), key=lambda x:x[1], reverse=True)
    punctuations_dict = {}
    punctuations_dict[ordered_dict[0][0]] = 10
    punctuations_dict[ordered_dict[1][0]] = 5
    punctuations_dict[ordered_dict[2][0]] = 2
    
    uni_proj = []
    for campo in selected_ids:
        percentage = uni_dict[int(campo)]
        uni_proj.append(punctuations_dict[int(campo)])
        
    return uni_proj

In [21]:
projs_dict = {}
for key, value in unis_dict.items():
    projs_dict[key] = calculate_uni_projection(value, selected_ids)
print(projs_dict)

{36: [5, 2, 10], 37: [2, 5, 10], 41: [2, 5, 10], 57: [5, 2, 10], 66: [2, 5, 10], 74: [2, 5, 10], 77: [2, 5, 10], 87: [5, 2, 10], 8: [5, 2, 10], 10: [2, 5, 10], 11: [5, 2, 10], 12: [2, 5, 10], 13: [-100, -100, -100], 14: [10, 2, 5], 16: [-100, -100, -100], 51: [10, 2, 5], 44: [5, 2, 10], 18: [5, 2, 10], 2: [-100, -100, -100], 3: [5, 2, 10], 22: [5, 2, 10], 24: [5, 2, 10], 21: [-100, -100, -100], 23: [-100, -100, -100], 26: [5, 2, 10], 5: [-100, -100, -100], 30: [5, 2, 10], 32: [2, 10, 5], 29: [-100, -100, -100], 33: [5, 2, 10], 48: [5, 2, 10], 34: [5, 2, 10], 20: [5, 2, 10], 35: [-100, -100, -100], 9: [2, 5, 10], 52: [2, 5, 10], 38: [5, 2, 10], 39: [10, 2, 5], 42: [5, 2, 10], 43: [5, 2, 10], 45: [2, 5, 10], 46: [5, 2, 10], 47: [2, 5, 10], 49: [2, 5, 10], 17: [5, 2, 10], 15: [2, 5, 10], 50: [5, 2, 10], 40: [5, 2, 10], 28: [2, 5, 10], 53: [2, 5, 10], 54: [5, 2, 10], 55: [-100, -100, -100], 56: [5, 2, 10], 58: [10, 2, 5], 61: [2, 5, 10], 59: [10, 2, 5], 60: [5, 2, 10], 62: [5, 2, 10], 63: 

In [22]:
def get_three_most_similar_unis(projs_dict, proj_user):
    """
        Searches for the three most similar unis to the user preferences
    """
    similarities_dict = {}
    for uni_key, proj in projs_dict.items():
        similarities_dict[uni_key] = 1 - spatial.distance.cosine(proj_user, proj)
        
    similarities_ordered = sorted(similarities_dict.items(), key=lambda x:x[1], reverse=True)
    return similarities_ordered[:3]

In [23]:
sims = get_three_most_similar_unis(projs_dict, proj_usuario)
print(sims)

[(14, 0.9302325581395349), (51, 0.9302325581395349), (39, 0.9302325581395349)]


In [13]:
print(sentencia_sql)

SELECT proyecto_final.universidades.nombre AS nombre_univ, proyecto_final.campos.nombre AS nombre_campo, proyecto_final.rankings.id_univ, proyecto_final.rankings.id_campo, proyecto_final.rankings.puntuacion FROM proyecto_final.campos
                    INNER JOIN proyecto_final.rankings ON proyecto_final.campos.id = proyecto_final.rankings.id_campo
                     INNER JOIN proyecto_final.universidades ON proyecto_final.universidades.id = proyecto_final.rankings.id_univ
                    WHERE proyecto_final.campos.id IN (4,5,11)


In [14]:
SELECT universidades.nombre AS nombre_univ, campos.nombre AS nombre_campo, rankings.id_univ, rankings.id_campo, rankings.puntuacion FROM campos INNER JOIN rankings ON campos.id = rankings.id_campo INNER JOIN universidades ON universidades.id = rankings.id_univ WHERE campos.id IN (4,5,11)

SyntaxError: invalid syntax (198619458.py, line 1)